# Create video from frames
This notebook combines segmentation labels and an image to show how well 

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image
import cv2
import skimage
from skimage.color import label2rgb
import tifffile as tf
import os
import pandas as pd

#import napari
#print(napari.__version__)

import io
import useful_functions as uf
#import numba_funcs as nf


In [20]:
from functools import partial
from copy import deepcopy
from tqdm import trange, tqdm
from numba import njit, jit, prange
#from microfilm import colorify #the pip version seems to be out of date
from datetime import datetime
from importlib import reload
from matplotlib_scalebar.scalebar import ScaleBar

In [21]:
# you may need to comment in the following line if you do not have moviepy installed
# !pip install moviepy
import moviepy
from moviepy.video.io.ImageSequenceClip import ImageSequenceClip

In [22]:
mycolors = uf.return_color_scale('block_colors_for_labels_against_gray',
                     show=False)*1000
mycolors[0] = 'black'

In [23]:
@njit
def get_edges_of_cluster_shapes_with_image_edges_int_output(intlabels, rgblabels, k = 2):
    """
    This function takes labels in two formats and returns the 
    label edges as a k-pixel edge image

    Parameters
    ----------
    intlabels : numpy array of ints with bg=0
        Labels in the format of an image with a different int for
        each label
    rgblabels : numpy array of floats RGB
        after from skimage.color import label2rgb has been called 
        on intlabels, with no image behind it
    k : int
        k is basically the thickness of the edging

    Returns
    -------
    output_edges 
        the image of output edges between the labels as int output

    """
    
    shape = intlabels.shape
    output_edges = np.zeros(shape, intlabels.dtype)
    for i in range(shape[0]):
        for j in range(shape[1]):
            this_val = intlabels[i,j]
            if this_val == 0:
                continue
            else:
                if i<k or shape[0]-i<k or j<k or shape[1]-j<k:
                    output_edges[i,j] = intlabels[i,j]
                else:
                    compare_array = intlabels[i-k:i+k+1,j-k:j+k+1]
                    allsame = np.all(compare_array == this_val)
                    
                    if allsame:
                        continue
                    else:
                        output_edges[i,j] = intlabels[i,j]
    return output_edges

In [24]:
@njit
def convert_rgb_to_rgba(img, transparent=np.array([0.,0.,0.]), transparency_val = int(1)):
    """
    Converts rgb to rgba while setting exactly one colur to fully transparent
    and preserving the rest.
    
    There is a useful func version of this that just sets one transparency
    
        To display properly with napari, you will also need to run
    convert_rgb_to_rgba, with 
     transparent=np.array([0,0,0]), transparency_val = int(255)
     Then blending translucent
          Showing stuff clearly on napari also seems to require the background 
     to be =1 not =0

    Parameters
    ----------
    img : numpy array (of floats i think, rgb)
        DESCRIPTION.
    transparent : numpy array of a 3 vec rgb, optional
        This is the colour which becomes transparent
        The default is np.array([0.,0.,0.]), which is black.
    transparency_val is for the non-transparent values

    Returns
    -------
    rgba_image : numpy array like img 
        RGBA version of img

    """
    shape = img.shape
    rgba_image = np.zeros((shape[0], shape[1], 4), img.dtype)
    for i in range(shape[0]):
        for j in range(shape[1]):
            if np.array_equal(img[i,j], transparent):
                rgba_image[i,j, 0:4] = 0
            else:
                rgba_image[i,j, 0:3] = img[i,j]
                rgba_image[i,j,3] = transparency_val
    return rgba_image

In [ ]:
filepath_in = os.path.abspath('../MorphoDynamicsPipe/1_data/fieldofview1/')
filenames_in = os.listdir(filepath_in)
filenames_in.sort(key=partial(uf.getvaluefromstringbest, variable='_T=', preceding='', ending='.', mydtype=int))
filenames_in[0:4]

filepath_in_labels = filepath_in.replace('1_data', '3c_tracking_images_filtered')
filepath_in_labels

In [26]:
directory_frames = 'frames/'
os.makedirs(directory_frames, exist_ok=True)

In [ ]:
for this_filename in tqdm(filenames_in):
    im = skimage.io.imread(os.path.join(filepath_in, this_filename))
    labels = skimage.io.imread(os.path.join(filepath_in_labels, this_filename))
    outlines = get_edges_of_cluster_shapes_with_image_edges_int_output(labels, label2rgb(labels), k=2) 

    x = skimage.util.img_as_ubyte(uf.label2rgb_with_dict(labels, mycolors))
    y = convert_rgb_to_rgba(x, transparent = np.array([0,0,0], dtype=np.uint8), transparency_val=255)

    x2 = skimage.util.img_as_ubyte(uf.label2rgb_with_dict(outlines, mycolors))
    y2 = convert_rgb_to_rgba(x2, transparent = np.array([0,0,0], dtype=np.uint8), transparency_val=255)

    plt.imshow(im, cmap='Greys_r', vmax=50)
    plt.imshow(y, alpha=0.2, interpolation='none')
    plt.imshow(y2, alpha=1, interpolation='none')

    plt.axis('off')
    scalebar = ScaleBar(0.57, "um", fixed_value=20, location='lower right',
                       font_properties={"size": 4})
    plt.gca().add_artist(scalebar)
    
    plt.savefig(os.path.join(directory_frames, this_filename),
               dpi=300, bbox_inches='tight', pad_inches=0)

    plt.show()
    
#    break

# Convert to video

In [ ]:
filename_out = 'video_output.avi'

In [ ]:
filenames_in = os.listdir(directory_frames)
filenames_in.sort(key=partial(uf.getvaluefromstringbest, variable='_T=', preceding='', ending='.', mydtype=int))
fullpaths = [os.path.join(directory_frames, file) for file in filenames_in]
fullpaths[0:2]

In [ ]:
#clip = ImageSequenceClip(list_of_images, fps=5)
clip = ImageSequenceClip(fullpaths, fps=2)

In [ ]:
clip.write_videofile(filename_out.replace('.avi', '.avi'), codec='png')
#clip.write_videofile(directory_out + filename_out.replace('.avi', '.mp4'), codec='mpeg4')